## CESM2 - LARGE ENSEMBLE (LENS2)

- This notebooks serves as an example on how to extract surface (or any other 2D spatial field) properties from a selected spacial region accross all LENS2 members for the atmosphere component. 

## Imports

In [1]:
import intake
import intake_esm
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import fsspec
import cmocean
import cartopy
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pop_tools
import sys
from distributed import Client
from ncar_jobqueue import NCARCluster
sys.path.append('../functions')
import util
from cartopy.util import add_cyclic_point
from misc import get_ij
import warnings, getpass, os

<div class="alert alert-block alert-info">
<b>Note:</b> comment the following line when debugging
</div>

In [2]:
warnings.filterwarnings("ignore")

### Dask workers

In [3]:
mem_per_worker = 100 # in GB 
num_workers = 80 
cluster = NCARCluster(cores=4, processes=3, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=6:mem={mem_per_worker}GB')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

<Client: 'tcp://10.12.206.54:44173' processes=0 threads=0, memory=0 B>


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/42492/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/42492/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.54:44173,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/42492/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Data Ingest

In [4]:
%%time
catalog = intake.open_esm_datastore(
    '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
)

CPU times: user 28.2 s, sys: 1.96 s, total: 30.2 s
Wall time: 31.8 s


In [5]:
catalog.search(component='atm').unique('frequency')

{'frequency': {'count': 5,
  'values': ['day_1', 'month_1', 'hour_6', 'hour_3', 'hour_1']}}

### Let's search for variables with montly frequency

In [6]:
cat_subset = catalog.search(component='atm',
                            frequency='month_1',
                            variable=['SHFLX','LHFLX','SST','PRECT','OCNFRAC','T']) # Sensible Heat Flux, Latent Heat Flux, Sea Surface Temperature, Total (Convective and large-scale) precipitation rate, Temperature

In [ ]:
%%time
dset_dict_raw = cat_subset.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


In [ ]:
# print keys
[key for key in dset_dict_raw.keys()]

In [ ]:
fb=('T','LHFLX','SHFLX','SST','PRECT','OCNFRAC')
pd=('historical','ssp370')
ff=('cmip6','smbb')
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    for iff in range(0,len(ff)):
        for ipd in range(0,len(pd)):
            str=f'ds_{pd[ipd]}_{ff[iff]}_{fb[ifb]} = dset_dict_raw[\'atm.{pd[ipd]}.cam.h0.{ff[iff]}.{fb[ifb]}\']'; exec(str)
        str=f'ds_{ff[iff]}_{fb[ifb]}=xr.combine_nested([ds_{pd[0]}_{ff[iff]}_{fb[ifb]},ds_{pd[1]}_{ff[iff]}_{fb[ifb]}],concat_dim=[\'time\'])'; exec(str)
        str=f'del ds_{pd[0]}_{ff[iff]}_{fb[ifb]},ds_{pd[1]}_{ff[iff]}_{fb[ifb]}'; exec(str) 
    str=f'ds_{fb[ifb]}=xr.combine_nested([ds_{ff[0]}_{fb[ifb]},ds_{ff[1]}_{fb[ifb]}],concat_dim=[\'member_id\'])'; exec(str)
    str=f'del ds_{ff[0]}_{fb[ifb]}, ds_{ff[1]}_{fb[ifb]}'; exec(str)
print(f'Done!')

### Mask

#### 1. Replace the SST data equal to 0 (continents) by NaN

In [ ]:
ds_SST.SST.isel(time=0,member_id=0).plot()

In [ ]:
ds_SST=ds_SST.where(ds_SST['SST'] != 0.)
ds_SST.SST.isel(time=0,member_id=0).plot()

#### 2. Building the mask. The ocean model and the atmospheric model feed data into the coastal region, so we need to take data that is 100% on the ocean model grid to ensure that we are not looking at data over the continent.  

In [ ]:
ds_OCNFRAC.OCNFRAC.isel(time=0,member_id=0).plot()

##### 2.2 Since we are not working with polar regions, we will put NAN on all data that is different from 1 to build the mask

In [ ]:
ds_OCNFRAC=ds_OCNFRAC.where(ds_OCNFRAC['OCNFRAC'] ==1.)
ds_OCNFRAC.OCNFRAC.isel(time=0,member_id=0).plot()

In [ ]:
mask_ocean = 2 * np.ones((ds_OCNFRAC.dims['lat'], ds_OCNFRAC.dims['lon'])) * np.isfinite(ds_OCNFRAC.OCNFRAC.isel(time=0,member_id=0))  
mask_land = 1 * np.ones((ds_OCNFRAC.dims['lat'], ds_OCNFRAC.dims['lon'])) * np.isnan(ds_OCNFRAC.OCNFRAC.isel(time=0,member_id=0))  
mask_array = mask_ocean + mask_land
mask_array.plot()

#### 3. Applying the mask for the other variables

In [ ]:
ds_PRECT =ds_PRECT.where(mask_array == 2.) # Precipitation 
ds_LHFLX =ds_LHFLX.where(mask_array == 2.) # Latent Heat Flux
ds_SHFLX =ds_SHFLX.where(mask_array == 2.) # Sensible Heat Flux
ds_T =ds_T.where(mask_array == 2.)         # Atmosphere Temperature
ds_SST =ds_SST.where(mask_array == 2.)     # Sea Surface Temperature

### Just to make sure that the mask was applied

In [ ]:
ds_PRECT.PRECT.isel(time=1,member_id=10).plot() # Precipitation (m/s)

In [ ]:
ds_LHFLX.LHFLX.isel(time=1,member_id=10).plot() # Latent Heat Flux (W/m2)

In [ ]:
ds_SHFLX.SHFLX.isel(time=1,member_id=10).plot() # Sensible Heat Flux (W/m2)

In [ ]:
ds_SST.SST.isel(time=1,member_id=10).plot()    # Sea Surface Temperature (K)

In [ ]:
ds_T.T.isel(time=1,member_id=10,lev=-1).plot() # Atmosphere Temperature (K)

### One more example...

In [ ]:
ds_var_1231 = ds_SHFLX.SHFLX.sel(member_id=['r1i1231p1f1','r2i1231p1f1','r3i1231p1f1','r4i1231p1f1','r5i1231p1f1','r6i1231p1f1','r7i1231p1f1','r8i1231p1f1','r9i1231p1f1','r10i1231p1f1']).mean(dim='time')#.plot()
plt.figure(figsize=(10,6));
ax = plt.axes(projection=ccrs.Robinson());
pc = ds_var_1231.mean(dim='member_id').plot.pcolormesh(ax=ax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmocean.cm.balance,
                    x='lon',
                    y='lat',
                    #vmin=-3,
                    #vmax=30,
                    cbar_kwargs={"orientation": "horizontal"})                                    
ax.gridlines(draw_labels=True);
ax.coastlines()
ax.gridlines();
plt.title("i1231p1f1")

### Centralize the South Atlantic 
Need to combine the domain in the east/west direction to centralize the South Atlantic

In [ ]:
ilat, flat = 60, 96
ilon1, flon1, ilon2, flon2 = 245, 288, 0, 17
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'sa_ds_{fb[ifb]}=xr.combine_nested([[ds_{fb[ifb]}.isel(lat = slice(ilat,flat),lon = slice(ilon1,flon1)),ds_{fb[ifb]}.isel(lat = slice(ilat,flat),lon = slice(ilon2,flon2))]],concat_dim=[\'lat\',\'lon\'])'
    exec(str)
    str=f'sa_ds_{fb[ifb]}.coords[\'lon\'] = (sa_ds_{fb[ifb]}.coords[\'lon\'] + 180) % 360 - 180' # change the longitudes: -180 0 180
    exec(str)
    str=f'sa_ds_{fb[ifb]} = sa_ds_{fb[ifb]}.sortby(sa_ds_{fb[ifb]}.lon)'
    exec(str)

### Let's test

In [ ]:
%%time
plt.figure(figsize=(10,6));
ax = plt.axes(projection=ccrs.Robinson());
pc = sa_ds_LHFLX.LHFLX.isel(member_id=0).mean(dim='time').plot.pcolormesh(ax=ax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmocean.cm.balance,
                    x='lon',
                    y='lat',
                    #vmin=290,
                    #vmax=302,
                    cbar_kwargs={"orientation": "horizontal"})                                    
ax.gridlines(draw_labels=True);
ax.coastlines()
ax.gridlines();

In [ ]:
%%time
# Annual Mean
sa_ds_SHFLX=sa_ds_SHFLX.resample(time='1Y', closed='left').mean('time')
print('SHFLX')
sa_ds_LHFLX=sa_ds_LHFLX.resample(time='1Y', closed='left').mean('time')
print('LHFLX')
sa_ds_SST=sa_ds_SST.resample(time='1Y', closed='left').mean('time')
print('SST')
sa_ds_T=sa_ds_T.resample(time='1Y', closed='left').mean('time')
print('T')
sa_ds_PRECT=sa_ds_PRECT.resample(time='1Y', closed='left').mean('time')
print('PRECT')

### Perfom computations
Calculate area mean, min, max, and rms for the variables of the selected region

In [ ]:
%%time
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    # Mean
    str=f'var_mean_{fb[ifb]} = sa_ds_{fb[ifb]}.{fb[ifb]}.mean(dim=(\'lon\',\'lat\')).load()'              # Mean
    exec(str)
    print(f'var_mean_{fb[ifb]}')
    # Maximum
    str=f'var_max_{fb[ifb]} = sa_ds_{fb[ifb]}.{fb[ifb]}.max(dim=(\'lon\',\'lat\')).load()'                # Maximum
    exec(str)
    print(f'var_max_{fb[ifb]}')
    # Minimum
    str=f'var_min_{fb[ifb]} = sa_ds_{fb[ifb]}.{fb[ifb]}.min(dim=(\'lon\',\'lat\')).load()'                # Minimum
    exec(str)
    print(f'var_min_{fb[ifb]}')
    # RMS
    str=f'var_rms_{fb[ifb]} = np.sqrt((sa_ds_{fb[ifb]}.{fb[ifb]}**2).mean(dim=[\'lat\',\'lon\'])).load()' # Root Mean Square
    exec(str)
    print(f'var_rms_{fb[ifb]}')
print(f'Done!')

### TODO
Plot some time series to check calculations

### Merge data and save on disk

In [ ]:
units=('K','W/m2','W/m2','K','m/s')
long_name=('Air Temperature','Latent Heat Flux','Sensible Heat Flux','Sea Surface Temperature','Total (Convective and Large-Scale) Precipitation Rate (liq + ice)')
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'ds_out_{fb[ifb]} = xr.merge([var_rms_{fb[ifb]}.rename(\'{fb[ifb]}_rms\'),var_mean_{fb[ifb]}.rename(\'{fb[ifb]}_mean\'),var_max_{fb[ifb]}.rename(\'{fb[ifb]}_max\'),var_min_{fb[ifb]}.rename(\'{fb[ifb]}_min\')])'
    exec(str)
    str=f'del var_rms_{fb[ifb]}, var_mean_{fb[ifb]}, var_max_{fb[ifb]}, var_min_{fb[ifb]}'
    exec(str)
    str=f'ds_out_{fb[ifb]}.attrs[\'description\'] = \'{long_name[ifb]} ({fb[ifb]}) statistics for the South Atlantic (53.75W-20E and 33.455497S-0.471204S)\''
    exec(str)
    str=f'ds_out_{fb[ifb]}.attrs[\'units\'] = \'{units[ifb]}\''
    exec(str)
    str=f'ds_out_{fb[ifb]}.attrs[\'author\'] = \'Mauricio Rocha\''
    exec(str)
    str=f'ds_out_{fb[ifb]}.attrs[\'email\'] = \'mauricio.rocha@usp.br\''
    exec(str)

In [ ]:
# PRECIPITATION
fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
(ds_out_PRECT.PRECT_max.resample(time='1Y', closed='left').mean('time')*86400000).plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_PRECT.PRECT_max.resample(time='1Y', closed='left').mean('time').mean('member_id')*86400000).plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y')
(ds_out_PRECT.PRECT_max.resample(time='5Y', closed='left').mean('time').mean('member_id')*86400000).plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('PRECT Mean [mm/day]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
fig.tight_layout(pad=3)

# Mean
(ds_out_PRECT.PRECT_mean.resample(time='1Y', closed='left').mean('time')*86400000).plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_PRECT.PRECT_mean.resample(time='1Y', closed='left').mean('time').mean('member_id')*86400000).plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y')
(ds_out_PRECT.PRECT_mean.resample(time='5Y', closed='left').mean('time').mean('member_id')*86400000).plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('PRECT Mean [mm/day]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()

# Minimum
(ds_out_PRECT.PRECT_min.resample(time='1Y', closed='left').mean('time')*86400000).plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_PRECT.PRECT_min.resample(time='1Y', closed='left').mean('time').mean('member_id')*86400000).plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y')
(ds_out_PRECT.PRECT_min.resample(time='5Y', closed='left').mean('time').mean('member_id')*86400000).plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('PRECT Min [mm/day]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()

# root mean square
(ds_out_PRECT.PRECT_rms.resample(time='1Y', closed='left').mean('time')*86400000).plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_PRECT.PRECT_rms.resample(time='1Y', closed='left').mean('time').mean('member_id')*86400000).plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y')
(ds_out_PRECT.PRECT_rms.resample(time='5Y', closed='left').mean('time').mean('member_id')*86400000).plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('PRECT Error [mm/day]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()

plt.show()

In [ ]:
#### SEA SURFACE TEMPERATURE
fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
(ds_out_SST.SST_max.resample(time='1Y', closed='left').mean('time')-273.15).plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_SST.SST_max.resample(time='1Y', closed='left').mean('time').mean('member_id')-273.15).plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y')
(ds_out_SST.SST_max.resample(time='5Y', closed='left').mean('time').mean('member_id')-273.15).plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('SST Max [oC]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
fig.tight_layout(pad=3)

# Mean
(ds_out_SST.SST_mean.resample(time='1Y', closed='left').mean('time')-273.15).plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_SST.SST_mean.resample(time='1Y', closed='left').mean('time').mean('member_id')-273.15).plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y')
(ds_out_SST.SST_mean.resample(time='5Y', closed='left').mean('time').mean('member_id')-273.15).plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('SST Mean [oC]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()

# Minimum
(ds_out_SST.SST_min.resample(time='1Y', closed='left').mean('time')-273.15).plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_SST.SST_min.resample(time='1Y', closed='left').mean('time').mean('member_id')-273.15).plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y')
(ds_out_SST.SST_min.resample(time='5Y', closed='left').mean('time').mean('member_id')-273.15).plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('SST Min [oC]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()

# root mean square
(ds_out_SST.SST_rms.resample(time='1Y', closed='left').mean('time')-273.15).plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_SST.SST_rms.resample(time='1Y', closed='left').mean('time').mean('member_id')-273.15).plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y')
(ds_out_SST.SST_rms.resample(time='5Y', closed='left').mean('time').mean('member_id')-273.15).plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('SST Error [oC]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()

plt.show()

In [ ]:
#### LATENT HEAT FLUX
fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
ds_out_LHFLX.LHFLX_max.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_LHFLX.LHFLX_max.resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y')
(ds_out_LHFLX.LHFLX_max.resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('LHFLX Max [W/m2]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
fig.tight_layout(pad=3)

# Mean
ds_out_LHFLX.LHFLX_mean.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_LHFLX.LHFLX_mean.resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y')
(ds_out_LHFLX.LHFLX_mean.resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('LHFLX Mean [W/m2]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()

# Minimum
ds_out_LHFLX.LHFLX_min.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_LHFLX.LHFLX_min.resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y')
(ds_out_LHFLX.LHFLX_min.resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('LHFLX Min [W/m2]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()

# root mean square
ds_out_LHFLX.LHFLX_rms.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_LHFLX.LHFLX_rms.resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y')
(ds_out_LHFLX.LHFLX_rms.resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('LHFLX Error [W/m2]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()

plt.show()

In [ ]:
#### SENSIBLE HEAT FLUX

fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
ds_out_SHFLX.SHFLX_max.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_SHFLX.SHFLX_max.resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y')
(ds_out_SHFLX.SHFLX_max.resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('SHFLX Max [oC]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
fig.tight_layout(pad=3)

# Mean
ds_out_SHFLX.SHFLX_mean.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_SHFLX.SHFLX_mean.resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y')
(ds_out_SHFLX.SHFLX_mean.resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('SHFLX Mean [oC]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()

# Minimum
ds_out_SHFLX.SHFLX_min.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_SHFLX.SHFLX_min.resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y')
(ds_out_SHFLX.SHFLX_min.resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('SHFLX Min [W/m2]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()

# root mean square
ds_out_SHFLX.SHFLX_rms.resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_SHFLX.SHFLX_rms.resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y')
(ds_out_SHFLX.SHFLX_rms.resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('SHFLX Error [W/m2]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()

plt.show()

In [ ]:
#### AIR TEMPERATURE

fig, axes = plt.subplots(1, 4, figsize=(20, 8))
# Maximum
ds_out_T.T_max.isel(lev=-1).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[0],x="time",color='orange',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_T.T_max.isel(lev=-1).resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[0],x="time",alpha=0.3,color='r',linewidth=1,label='Member Mean 1Y')
(ds_out_T.T_max.isel(lev=-1).resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[0],x="time",color='r',linewidth=2,label='Member Mean 5Y')
axes[0].set_xlabel('Time [Years]')
axes[0].set_ylabel('SHFLX Max [W/m2]')
axes[0].set_title('Area Max')
axes[0].grid(color='k', linestyle='-', linewidth=0.7)
axes[0].legend()
fig.tight_layout(pad=3)

# Mean
ds_out_T.T_mean.isel(lev=-1).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[1],x="time",color='gray',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_T.T_mean.isel(lev=-1).resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[1],x="time",alpha=0.3,color='k',linewidth=1,label='Member Mean 1Y')
(ds_out_T.T_mean.isel(lev=-1).resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[1],x="time",color='k',linewidth=2,label='Member Mean 5Y')
axes[1].set_xlabel('Time [Years]')
axes[1].set_ylabel('SHFLX Mean [W/m2]')
axes[1].set_title('Area Mean')
axes[1].grid(color='k', linestyle='-', linewidth=0.7)
axes[1].legend()

# Minimum
ds_out_T.T_min.isel(lev=-1).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[2],x="time",color='c',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_T.T_min.isel(lev=-1).resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[2],x="time",alpha=0.3,color='b',linewidth=1,label='Member Mean 1Y')
(ds_out_T.T_min.isel(lev=-1).resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[2],x="time",color='b',linewidth=2,label='Member Mean 5Y')
axes[2].set_xlabel('Time [Years]')
axes[2].set_ylabel('SHFLX Min [W/m2]')
axes[2].set_title('Area Min')
axes[2].grid(color='k', linestyle='-', linewidth=0.7)
axes[2].legend()

# root mean square
ds_out_T.T_rms.isel(lev=-1).resample(time='1Y', closed='left').mean('time').plot.line(ax=axes[3],x="time",color='y',alpha=0.01,linewidth=1,add_legend=False)
(ds_out_T.T_rms.isel(lev=-1).resample(time='1Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[3],x="time",alpha=0.3,color='g',linewidth=1,label='Member Mean 1Y')
(ds_out_T.T_rms.isel(lev=-1).resample(time='5Y', closed='left').mean('time').mean('member_id')).plot.line(ax=axes[3],x="time",color='g',linewidth=2,label='Member Mean 5Y')
axes[3].set_xlabel('Time [Years]')
axes[3].set_ylabel('SHFLX Error [W/m2]')
axes[3].set_title('Area Error')
axes[3].grid(color='k', linestyle='-', linewidth=0.7)
axes[3].legend()

plt.show()

### Let's save the data in netcdf format

In [ ]:
# create a directory on scratch to save the output
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'path = \'/glade/scratch/mauricio/Data/LENS2/{fb[ifb]}/\'.format(getpass.getuser())'
    exec(str)
    str=f'os.system(\'mkdir -p \'+path)'
    exec(str)
    str=f'ds_out_{fb[ifb]}.to_netcdf(path+\'{fb[ifb]}_stats.nc\')'
    exec(str)

In [ ]:
cluster.close()
client.close()